### Setup

In [1]:
import numpy as np
import tensorflow as tf
import keras

import os


### Constants

In [2]:
EPOCH = 30
BATCH = 128

DATADIR = "/Users/mghifary/Work/Code/AI/data"
MODELDIR = "/Users/mghifary/Work/Code/AI/models"

### Create sampling layer

In [3]:
class Sampling(keras.layers.Layer):
    """
    Uses (z_mean, z_log_var) to sample z, the vector encoding a digit.
    """

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.random.normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

### Build the encoder

In [4]:
latent_dim = 2

encoder_inputs = keras.Input(shape=(28, 28, 1))
x = keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu", strides=2, padding="same")(encoder_inputs)
x = keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu", strides=2, padding="same")(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(16, activation="relu")(x)

z_mean = keras.layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = keras.layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])

encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 28, 28, 1)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 14, 14, 32)           320       ['input_1[0][0]']             
                                                                                                  
 conv2d_1 (Conv2D)           (None, 7, 7, 64)             18496     ['conv2d[0][0]']              
                                                                                                  
 flatten (Flatten)           (None, 3136)                 0         ['conv2d_1[0][0]']            
                                                                                            

### Build the decoder

In [5]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = keras.layers.Dense(7 * 7 * 64, activation="relu")(latent_inputs)
x = keras.layers.Reshape((7, 7, 64))(x)
x = keras.layers.Conv2DTranspose(64, kernel_size=(3, 3), activation="relu", strides=2, padding="same")(x)
x = keras.layers.Conv2DTranspose(32, kernel_size=(3, 3), activation="relu", strides=2, padding="same")(x)
decoder_outputs = keras.layers.Conv2DTranspose(1, kernel_size=(3, 3), activation="sigmoid", padding="same")(x)

decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 2)]               0         
                                                                 
 dense_1 (Dense)             (None, 3136)              9408      
                                                                 
 reshape (Reshape)           (None, 7, 7, 64)          0         
                                                                 
 conv2d_transpose (Conv2DTr  (None, 14, 14, 64)        36928     
 anspose)                                                        
                                                                 
 conv2d_transpose_1 (Conv2D  (None, 28, 28, 32)        18464     
 Transpose)                                                      
                                                                 
 conv2d_transpose_2 (Conv2D  (None, 28, 28, 1)         289 

### Build VAE

In [6]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_trakcer = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
    
    @property
    def metrics(self):
        return [
            self.total_loss_trakcer,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker
        ]
    
    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = encoder(data)
            reconstruction = decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=(1 , 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        
        self.total_loss_trakcer.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        
        return {
            "loss": self.total_loss_trakcer.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result()
        }

### Load dataset

In [7]:
data_path = os.path.join(DATADIR, "mnist.npz")
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data(data_path)
x_train = x_train.astype("float32") / 255.
x_test = x_test.astype("float32") / 255.

mnist_digits = np.concatenate([x_train, x_test], axis=0)
mnist_digits = np.expand_dims(mnist_digits, -1)

### Train VAE

In [8]:
model = VAE(encoder, decoder)
model.compile(optimizer=keras.optimizers.legacy.Adam(3e-4))
model.fit(mnist_digits, epochs=EPOCH, batch_size=BATCH)

Epoch 1/30
547/547 [==============================] - 16s 28ms/step - loss: 303.2931 - reconstruction_loss: 229.1234 - kl_loss: 4.3461
Epoch 2/30
547/547 [==============================] - 16s 29ms/step - loss: 192.9297 - reconstruction_loss: 183.8243 - kl_loss: 4.8164
Epoch 3/30
547/547 [==============================] - 16s 29ms/step - loss: 177.5176 - reconstruction_loss: 168.9827 - kl_loss: 5.7014
Epoch 4/30
547/547 [==============================] - 16s 30ms/step - loss: 169.1217 - reconstruction_loss: 162.6206 - kl_loss: 5.2397
Epoch 5/30
547/547 [==============================] - 17s 30ms/step - loss: 165.0161 - reconstruction_loss: 158.8882 - kl_loss: 5.2580
Epoch 6/30
547/547 [==============================] - 17s 31ms/step - loss: 162.1151 - reconstruction_loss: 156.1014 - kl_loss: 5.4126
Epoch 7/30
547/547 [==============================] - 17s 31ms/step - loss: 159.7864 - reconstruction_loss: 154.3065 - kl_loss: 5.5158
Epoch 8/30
547/547 [==============================] - 1

In [9]:
# Save model
encoderpath = os.path.join(MODELDIR, "vae-mnist-conv-encoder.h5")
encoder.save_weights(encoderpath, overwrite=True, save_format=None, options=None)

decoderpath = os.path.join(MODELDIR, "vae-mnist-conv-decoder.h5")
decoder.save_weights(decoderpath, overwrite=True, save_format=None, options=None)